In [1]:
import requests
import pandas as pd
import numpy as np
import time

In [2]:
# first account
def similarweb(website):
	url = "https://similarweb2.p.rapidapi.com/trafficoverview"
	querystring = {"website":"https://www."+website}

	headers = {
		"X-RapidAPI-Host": "similarweb2.p.rapidapi.com",
		"X-RapidAPI-Key": "b00f8a11c7msh7cb123610208633p1d2f21jsn969dea2cff92"
	}

	return requests.request("GET", url, headers=headers, params=querystring)

In [96]:
# second account
def similarweb(website):
	url = "https://similarweb2.p.rapidapi.com/trafficoverview"
	querystring = {"website":"https://www."+website}

	headers = {
		"X-RapidAPI-Host": "similarweb2.p.rapidapi.com",
		"X-RapidAPI-Key": "76cbb17fa7mshe73212cc89313edp1e13e1jsnc185fa496d5f"
	}

	return requests.request("GET", url, headers=headers, params=querystring)

In [172]:
# getting the list of websites
df_sources = pd.read_excel('Анализ медиа пространства_20220504.xlsx', header=1)
sites = df_sources['source'].to_list()

In [186]:
# collecting info
# sites = ['tvrain.ru'] # не подгрузившиеся сайты
df_traffic_all = []
for website in sites:
    # trying to get and read k times. if error try again.
    k = 6
    while k>0:
        try:
            response = similarweb(website) #info from SimilarWeb
            # collect history of visits
            df_traffic = pd.DataFrame({website:list(response.json()['monthlyVisitsEstimate'].values())}).T #often does not return proper repsonse (e.g., not enough data)
            df_traffic.columns = list(response.json()['monthlyVisitsEstimate'].keys())
            # add countries shares data
            param = 'trafficShareByCountry'
            for i in response.json()[param]:
                df_traffic = pd.concat([df_traffic, pd.DataFrame(i, index=[website])], axis=1)
            k = 0
            # append to list of all requests
            df_traffic_all.append(df_traffic)
        except:
            k -= 1
        time. sleep(1)

In [185]:
# join info on all sites into one dataframe
df = pd.concat(df_traffic_all)
df.iloc[:,:6] = df.iloc[:,:6]/1000
df.to_clipboard(excel=True)

In [173]:
pd.concat([df_sources.set_index('source'), df], axis=1)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [157]:
list(df.index).sort() == list(df_sources['source']).sort()

True

In [174]:
source_list1 = list(df_sources['source'])
source_list2 = list(df.index)

In [176]:
source_list1.sort()
source_list2.sort()

In [183]:
len(set(source_list1))

30

### experiments

In [27]:
website = 'meduza.io'
resp = similarweb(website)
resp.content

b'{"engagement":{"avgVisitDuration":234,"bounceRate":0.5549,"pagesPerVisit":2.36,"totalVisits":31829000},"monthlyVisitsEstimate":{"2021-11-01":25190000,"2021-12-01":25070000,"2022-01-01":27340000,"2022-02-01":39700000,"2022-03-01":48500000,"2022-04-01":31820000},"name":"meduza.io","trafficShareByCountry":[{"Russian Federation":0.4218},{"United States":0.1014},{"Ukraine":0.087},{"Germany":0.0485},{"Netherlands":0.0402}],"trafficSources":{"Direct":0.7071,"Mail":0.0116,"Paid Referrals":0.0001,"Referrals":0.0251,"Search":0.1853,"Social":0.0705}}\n'

In [68]:
param = 'trafficShareByCountry'
df=pd.DataFrame()
for i in resp.json()[param]:
    df = pd.concat([df, pd.DataFrame(i, index=[0])], axis=1)

In [69]:
df

,Russian Federation,United States,Ukraine,Germany,Netherlands
0,0.4218,0.1014,0.087,0.0485,0.0402


In [ ]:
param = 'engagement'
type(response.json()[param]), response.json()[param]

(dict,
 {'avgVisitDuration': 170,
  'bounceRate': 0.6133,
  'pagesPerVisit': 2.26,
  'totalVisits': 207820000})